# Load packages

In [1]:
# trimming related
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import re, string
from pprint import pprint
import time


# Link to GDrive

In [ ]:
# !pip install -U -q PyDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# if 2==1:
#   # 2. Create & upload a file text file.
#   uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
#   uploaded.SetContentString('Sample upload file content')
#   uploaded.Upload()
#   print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [ ]:
# #https://drive.google.com/open?id=19c1xhHZrla2JngnGDaBzGXsUhJyC7WWo

# if 1==1:
#   #2. Get the file
#   downloaded = drive.CreateFile({'id':'19c1xhHZrla2JngnGDaBzGXsUhJyC7WWo'}) # replace the id with id of file you want to access
#   downloaded.GetContentFile('combined_trainingdata_20181013.tsv')  

#   #3. Read file as panda dataframe
#   import pandas as pd
#   df = pd.read_csv('combined_trainingdata_20181108.tsv',sep='\t') 
#   df.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [2]:
# Import data
dat = pd.read_csv('combined_trainingdata_20181108.tsv', sep='\t')
dat.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

c:\users\alexl\envs\mlbase\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961399 entries, 0 to 961398
Data columns (total 11 columns):
index             961399 non-null int64
COMPLAINT_ID      961399 non-null object
CITY              961399 non-null object
COMPLAINT DATE    961399 non-null object
DEPT_311          629980 non-null object
CODE_311          961399 non-null object
CATEGORY_MAIN     961399 non-null object
CATEGORY_SUB      961399 non-null object
COMPLAINT_1       961399 non-null object
COMPLAINT_2       281047 non-null object
complaint         961399 non-null object
dtypes: int64(1), object(10)
memory usage: 80.7+ MB


# Verify data

In [3]:
dat.shape

(961399, 11)

In [8]:
dat.head(2)

,index,COMPLAINT_ID,CITY,COMPLAINT DATE,DEPT_311,CODE_311,CATEGORY_MAIN,CATEGORY_SUB,COMPLAINT_1,COMPLAINT_2,complaint
0,0,US_CHICAGO_1725,US_CHICAGO,10/03/2011,health_environment,permits issued by doe work order,environment,environment_general,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...,[INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...,questionable business practices regarding oils...
1,1,US_CHICAGO_600301602,US_CHICAGO,07/14/2017,health_environment,illegal dumping work order,environment,environment_dumping,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...,[INSPECTION LOG #: 3439804 14-JUL-17 09:45:00]...,constructiondemolition debris dumped back yard...


In [9]:
model_data=dat[['CATEGORY_MAIN','CATEGORY_SUB','complaint']]

In [10]:
model_data.shape

(961399, 3)

# Basic Modeling

In [12]:
# Load packages
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score

%matplotlib inline

** drop any category with less than 100 occurances**

In [14]:
model_data.dropna(axis=0,subset=['complaint'],inplace=True)
model_data = model_data[model_data.groupby('CATEGORY_SUB').CATEGORY_MAIN.transform(len) > 99]

In [15]:
model_data.shape

(961091, 3)

In [16]:
y_main= model_data['CATEGORY_MAIN']
y_sub= model_data['CATEGORY_SUB']
x= model_data['complaint']

count_vect = CountVectorizer()
x_counts = count_vect.fit_transform(x)
tfidf_transformer = TfidfTransformer()
x_tfidf = tfidf_transformer.fit_transform(x_counts)

In [17]:
#main categories
x_train_main, x_test_main, y_train_main, y_test_main = train_test_split( x_tfidf, y_main, test_size=0.2, random_state=412, stratify=y_main)

#sub categories
x_train_sub, x_test_sub, y_train_sub, y_test_sub = train_test_split( x_tfidf, y_sub, test_size=0.2, random_state=412, stratify=y_sub)

## Naive Bayes

In [18]:
modelNB_sub = MultinomialNB().fit(x_train_sub, y_train_sub)

In [19]:
display(modelNB_sub.predict(count_vect.transform(["fire escape"]))[0])
display(modelNB_sub.predict(count_vect.transform(["trash left in empty lot"]))[0])
display(modelNB_sub.predict(count_vect.transform(["broken window in abandoned house"]))[0])
display(modelNB_sub.predict(count_vect.transform(["graffiti"]))[0])

'housing_general'

'environment_garbage_collection'

'environment_abandoned_vehicle'

'vandalism_graffiti'

In [20]:
y_predict_NB_sub=modelNB_sub.predict(x_test_sub)

In [21]:
print('accuracy score:',accuracy_score(y_test_sub, y_predict_NB_sub, normalize=True, sample_weight=None))

print(classification_report(y_test_sub, y_predict_NB_sub))

accuracy score: 0.6994261753520724


c:\users\alexl\envs\mlbase\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.00      0.00      0.00        54
     environment_abandoned_site       1.00      0.17      0.29       833
  environment_abandoned_vehicle       0.77      0.71      0.74      5043
      environment_air_pollution       0.68      0.91      0.78      3551
           environment_asbestos       1.00      0.04      0.07       542
        environment_dead_animal       0.93      0.59      0.73      1972
            environment_dumping       0.59      0.53      0.56      7219
 environment_garbage_collection       0.75      0.98      0.85     47408
            environment_general       0.95      0.06      0.11      1511
 environment_hazardous_material       0.92      0.20      0.32      1151
             environment_litter       0.98      0.15      0.26      2982
         environment_overgrowth       0.56      0.91      0.69     15871
          environment_recycling       0.97      0.

## SVM

In [22]:
modelsvc_sub = LinearSVC()
modelsvc_sub.fit(x_train_sub, y_train_sub)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [23]:
display(modelsvc_sub.predict(count_vect.transform(["fire escape"]))[0])
display(modelsvc_sub.predict(count_vect.transform(["trash left in empty lot"]))[0])
display(modelsvc_sub.predict(count_vect.transform(["broken window in abandoned house"]))[0])
display(modelsvc_sub.predict(count_vect.transform(["graffiti"]))[0])

'fire_general'

'environment_garbage_collection'

'housing_general'

'vandalism_graffiti'

In [24]:
y_predict_svc_sub = modelsvc_sub.predict(x_test_sub)

In [25]:
y_predict_svc_sub = pd.DataFrame(y_predict_svc_sub, dtype='str',columns=['predict_SUB'])['predict_SUB'].map(lambda x: x.replace('environemnt','environment'))

In [26]:
print('accuracy score:',accuracy_score(y_test_sub, y_predict_svc_sub, normalize=True, sample_weight=None))

print(classification_report(y_test_sub, y_predict_svc_sub))

accuracy score: 0.8115742980662681
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.00      0.00      0.00        54
     environment_abandoned_site       0.70      0.53      0.60       833
  environment_abandoned_vehicle       0.76      0.85      0.80      5043
      environment_air_pollution       0.85      0.89      0.87      3551
           environment_asbestos       0.91      0.94      0.93       542
        environment_dead_animal       0.85      0.93      0.89      1972
            environment_dumping       0.67      0.64      0.65      7219
 environment_garbage_collection       0.89      0.96      0.92     47408
            environment_general       0.65      0.41      0.50      1511
 environment_hazardous_material       0.78      0.73      0.76      1151
             environment_litter       0.69      0.50      0.58      2982
         environment_overgrowth       0.78      0.88      0.82     15871
          envir

### let's convert to main category 
**and see how they do**

In [ ]:
y_predict_svc_sub[:2]

In [ ]:
y_dummy=y_test_sub.map(lambda x: x.split('_')[0])


predict_dummy= y_predict_svc_sub.map(lambda x: x.split('_')[0])

In [ ]:
print('accuracy score:',accuracy_score(y_dummy, predict_dummy, normalize=True, sample_weight=None))

print(classification_report(y_dummy, predict_dummy))

In [ ]:
#with MAIN classes
cm_svc = confusion_matrix(y_dummy, predict_dummy)

fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cm_svc, annot=True, fmt='d')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()
display(accuracy_score(y_dummy, predict_dummy, normalize=True, sample_weight=None))

## Random Forest

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
modelRf_sub = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
modelRf_sub.fit(x_train_sub, y_train_sub)

In [ ]:
y_predict_rf_sub=modelRf_sub.predict(x_test_sub)

In [ ]:
y_predict_rf_sub = pd.DataFrame(y_predict_rf_sub, dtype='str',columns=['predict_SUB'])['predict_SUB'].map(lambda x: x.replace('environemnt','environment'))

In [ ]:
print('accuracy score:',accuracy_score(y_test_sub, y_predict_rf_sub, normalize=True, sample_weight=None))

print(classification_report(y_test_sub, y_predict_rf_sub))

# Multiclass Logistic Regression
- Max entropy for multiclass  
- Regularization = L2 (only one avail for max ent)  
- Class weight = 'none' by default, but setting to 'balanced' boosts rare classes


In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
start_time = time.time()
model_lr_sub = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial', max_iter=500)

model_lr_sub.fit(x_train_sub, y_train_sub)
print(f'Time elapsed: {time.time() - start_time}')

Time elapsed: 1733.272845029831


c:\users\alexl\envs\mlbase\lib\site-packages\sklearn\linear_model\logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [29]:
start_time = time.time()
model_lr_sub = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial', max_iter=500)

model_lr_sub.fit(x_train_sub, y_train_sub)
print(f'Time elapsed: {time.time() - start_time}')

Time elapsed: 1733.272845029831


c:\users\alexl\envs\mlbase\lib\site-packages\sklearn\linear_model\logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [30]:
start_time = time.time()

y_predict_lr_sub=model_lr_sub.predict(x_test_sub)

print(f'Time elapsed: {time.time() - start_time}')

Time elapsed: 0.2523622512817383


In [31]:
print('accuracy score:',accuracy_score(y_test_sub, y_predict_lr_sub, normalize=True, sample_weight=None))

print(classification_report(y_test_sub, y_predict_lr_sub))

accuracy score: 0.8112777613035131


c:\users\alexl\envs\mlbase\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.86      0.11      0.20        54
     environment_abandoned_site       0.75      0.54      0.62       833
  environment_abandoned_vehicle       0.77      0.84      0.81      5043
      environment_air_pollution       0.85      0.88      0.87      3551
           environment_asbestos       0.94      0.90      0.92       542
        environment_dead_animal       0.87      0.91      0.89      1972
            environment_dumping       0.68      0.66      0.67      7219
 environment_garbage_collection       0.88      0.96      0.92     47408
            environment_general       0.68      0.43      0.52      1511
 environment_hazardous_material       0.80      0.72      0.76      1151
             environment_litter       0.68      0.51      0.58      2982
         environment_overgrowth       0.78      0.87      0.82     15871
          environment_recycling       0.94      0.

# Ensemble Soft Vote
#### Linear regression:  
- multiclass = 'multinomial' # max entropy  
- regularization = 'l2' # Only one available for max ent

### main categories
x_train_main, x_test_main, y_train_main, y_test_main = `train_test_split( x_tfidf, y_main, test_size=0.2, random_state=412, stratify=y_main)`

### sub categories
x_train_sub, x_test_sub, y_train_sub, y_test_sub = `train_test_split( x_tfidf, y_sub, test_size=0.2, random_state=412, stratify=y_sub)`

In [ ]:
from sklearn.ensemble import VotingClassifier

start_time = time.time()

clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial',
                          random_state=1)
clf2 = RandomForestClassifier(n_estimators=100, random_state=1)
clf3 = MultinomialNB()
clf4 = LinearSVC()


eclf_soft = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('mnb', clf3), ('lsvc', clf4)], voting='soft')

eclf_soft = eclf_soft.fit(x_train_sub, y_train_sub)

print(f'Time elapsed: {time.time() - start_time}')

In [ ]:
start_time = time.time()
y_predict_eclf_sub = eclf_soft.predict(x_test_sub)
print(f'Time elapsed: {time.time() - start_time}')